In [3]:
import numpy as np
import EFIE as solve
light_speed, mu0, eps0 = 299792458., 4*np.pi*1e-7, 8.854e-12

frequency = 1e6 * 146
omega, wavelength = 2 * np.pi * frequency, light_speed / frequency
incident_field, radius = 220, 3.175e-3
delta_r = wavelength / 100

class yagi:
    def __init__(self, position, angle, length, source_position, radius):
        self.position = position
        self.angle = angle
        self.length = length
        self.source_position = source_position
        self.radius = radius

class tree:
    def __init__(self, its, phi, length, f, radius, field):
        self.its = its
        self.phi = phi
        self.length = length
        self.f = f
        self.radius = radius
        self.field = field

basis_functions = 'triangle'
structure_type = 'yagi-uda'
position = np.array([[0.0, 0.0, 0.0]])
angle = np.array([[np.pi/2, np.pi/4]])
length = np.array([delta_r*5])
source_position = np.array([[0.0, 0.0, 0.0, incident_field]])
radii = np.array([radius])
test_yagi = yagi(position, angle, length, source_position, radii)

In [ ]:
I,R = solve.calc_current_amplitudes(structure_type,basis_functions,test_yagi,frequency,delta_r)

In [ ]:
E,angles = solve.calc_field_pattern(0,0,basis_functions,structure_type,test_yagi,I,R,delta_r,frequency)
print(np.abs(E))

In [ ]:
import numpy as np 
from scipy import linalg 
import scipy.integrate as integrate 
from tqdm import tqdm
import geometry as gm
import matrix_elements as matrix_elements

# define constants

light_speed, mu0, eps0 = 299792458., 4*np.pi*1e-7, 8.854e-12
c = light_speed

def basis_func(basis_functions, t_n, r_n, dr_n):
    if basis_functions == 'pulse' :
        return 1
    elif basis_functions == 'triangle':
        if -1/2 <= t_n <= 1/2 :
            return 1/2 + t_n
        elif 1/2 <= t_n <= 3/2 :
            return 3/2 - t_n

def weight_func(basis_functions, t_m, r_m, dr_m):
    if basis_functions == 'pulse' :
        return 1
    elif basis_functions == 'triangle':
        if -1/2 <= t_m <= 1/2 :
            return 1/2 + t_m
        elif 1/2 <= t_m <= 3/2 :
            return 3/2 - t_m

def exp_dp_real (t_m, point, r_m, dr_m, omega, basis_functions):
    k = omega/c
    rmn = np.linalg.norm(point - r_m - dr_m * (t_m - 1/2))
    f_m = weight_func(basis_functions=basis_functions, t_m=t_m, r_m=r_m, dr_m=dr_m)
    return f_m * np.cos(-k * rmn)

def exp_dp_imag (t_m, point, r_m, dr_m, omega, basis_functions):
    k = omega/c
    rmn = np.linalg.norm(point - r_m - dr_m * (t_m - 1/2))
    f_m = weight_func(basis_functions=basis_functions, t_m=t_m, r_m=r_m, dr_m=dr_m)
    return f_m * np.sin(-k * rmn)

def calc_field_pattern_new (phi, theta, basis_functions, structure_type, antenna, current, R, delta_r, frequency) :

    omega = 2 * np.pi * frequency
    
    v = np.array([np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)])
    aux_v = np.array([1, 0, 0])
    perp_v1 = np.cross(v, aux_v) / np.linalg.norm(np.cross(v, aux_v))
    perp_v2 = np.cross(perp_v1, v) / np.linalg.norm(np.cross(perp_v1, v))

    Radius, num_points = 1e2, 100
    angles = np.linspace(0, 2 * np.pi, num_points)

    points = np.zeros((num_points, 3))
    for i, angle in enumerate(angles):
        points[i] = Radius * (np.cos(angle) * perp_v1 + np.sin(angle) * perp_v2)

    segments_block, source_segments = gm.antenna_to_segments(structure_type='yagi-uda', antenna=antenna, basis_functions='triangle', delta_r=delta_r)
    dr_n = delta_r * segments_block[0][0].tau
    if basis_functions == 'pulse_optimized':
        E = []
        k = 2 * np.pi * frequency / light_speed
        for point in points:
            E_i = 0
            for j in range(len(R)):
                rmn = np.linalg.norm(point - R[j])
                k_vec = point - R[j] / rmn
                k_p = np.dot(k_vec, dr_n)
                if k_p == 0:
                    E_i += - 1j*omega*mu0*current[j]*delta_r/(2*np.pi*rmn) * np.exp(-1j*k*rmn)
                else:
                    E_i += - 1j* omega * mu0 * current[j] * delta_r / (4 * np.pi * rmn * k_p**2) * np.exp(-1j*k*rmn) * np.exp(-1j*k_p)*(2*np.exp(1j*k_p) - np.exp(2j*k_p)-1)
            E.append(E_i)
        E_total = np.abs(E)
        return E_total, angles
    
    
    return E_total, angles

In [ ]:
E_new, angles = calc_field_pattern_new(0,0,'pulse_optimized', structure_type,test_yagi,I,R,delta_r,frequency)

In [ ]:
print((E_new/E))

In [ ]:
import matplotlib.pyplot as plt 
plt.polar(angles,E, label = 'exact solution')
plt.polar(angles,E_new, label = 'analit')
plt.legend()

In [4]:
import geometry as gm
import matrix_elements as matrix_elements

segments_block, source_segments = gm.antenna_to_segments(structure_type=structure_type, antenna=test_yagi, basis_functions=basis_functions, delta_r=delta_r)
impedance = matrix_elements.calculate_impedance(basis_functions=basis_functions, structure_type=structure_type, segments_block=segments_block, frequency=frequency, delta_r=delta_r)